## Global Imports

In [7]:
import pandas as pd
import numpy as np

## Function used in Problem 2 - Question 4

In [8]:
def perceptron(X1, X2, target1, target2, epochs):
    """
      Perceptron algorithm for finding the separating hyperplane between class X1 (with target1) and class X2 (with target target2).
      We set a maximum number of epochs for the algorithm to run if the algorithm fails to terminate on its own. The algorithm
      terminates on its own if there are no misclassifications, ie the hyperplane perfectly separates the two classes.
      We measure the misclassifications (or equivalently the lack of them) by the times we update the weights (or equivalenty don't update them) in an epoch.
      If the weights don't get updated even one time in an epoch, the algorithm has converged to a solution that perfectly separates the two classes.
    """

    # Multiplies the samples of each class with the corresponding target and creates matrix X containing the samples of both classes.
    X = np.concatenate((X1*target1, X2*target2), axis=0)

    # Zero initial weights.  
    w = np.zeros(X.shape[1])
    # Parameters for measuring the amount of misclassifications and the current epoch.
    count = 0
    epoch = 0
    
    # Loop while we are under the given number of epochs and we have some misclassifications.
    while epoch < epochs and count!=len(X):
        #print(f"Epoch {epoch}")
        count = 0

        for i in range(len(X)):
            # Condition for updating weights: t*w*x <= 0
            cond = np.dot(X[i], w)

            if cond<=0:
                w += X[i]
                #print(f"Cond {cond} Update Y {X[i]} -> {w}")
            else:
                count += 1
                #print(f"Cond {cond} Update N {X[i]}")
                
        epoch += 1
    #print(f"\nFinal w: {w}")
    return w, epoch, count

## Problem 2 - Question 6

In this question, we implemented the Perceptron algorithm on the IRIS PLANT DATABASE to perform classification on the 3 classes. Specifically, we examined if each class is linearly separable from the two other classes.

Preprocessing: After loading the dataset, we splitted it to 2 classes (one class containing one type of plant and the other "super" class containing the other two classes). We modified every sample/pattern in each class, by appending a feature of value -1 to the end of it. This feature corresponds to the threshold $w_0$ of the weight vector. Also, we adopted target outputs $+1$ for one class and $-1$ for the other, multiplying the samples of each class by the target.

Algorithm: We started by initial synaptic weights $\vec{w}=(w_1,w_2,...,w_0)$ set to 0. At every epoch, we presented the network every pattern $\vec{x_k}$ in the two classes and we calculated $t\cdot \vec{w}\vec{x_k}=\vec{w}\vec{x_k'}$ with the current weights value. If $\vec{w}\vec{x'_k}\leq 0$ we updated the weights by the rule $\vec{w} \rightarrow \vec{w}+\Delta \vec{w}=\vec{w}+\vec{w}\vec{x_k'}$. 

The epoch ends after all patterns are shown to the network one time. If we didn't update the weights at a given epoch, the algorithm has converged and there are no misclassifications (points classified on the wrong class / on the wrong side on the hyperplane). 

The separating hyperplane after convergence or after exceeding max number of epochs (given by user) is $\vec{w}\cdot \vec{x}-w_0=0\rightarrow w_1\cdot x_1+w_2\cdot x_2 +...+w_n\cdot x_n - w_0=0$

*To show that the perceptron we developed is working correctly, we ran the algorithm on two of the examples conducted during the lectures (The first one in 'PERCEPTRON.pdf' and the second one in 'notes_lecture_8.pdf').


In [9]:
# Examples from class. We've already added the -1 feature corresponding to w0.
# Example from PERCEPTRON.pdf
X1 = np.array([[1, -1, -1],
               [-1, 1, -1],
               [1, 1, -1]])
X2 = np.array([[-1, -1, -1]])

w, epoch, count = perceptron(X1, X2, 1, -1, 10)
print(f"Final weights: {w}  At epoch: {epoch} Number of correctly classified patterns: {count}")

# Example from lecture notes.
X1 = np.array([[1, 0, -1],
              [0, 1, -1]])
X2 = np.array([[-1, 0, -1]])

w, epoch, count = perceptron(X1, X2, 1, -1, 10)
print(f"Final weights: {w}  At epoch: {epoch} Number of correctly classified patterns: {count}")

Final weights: [ 1.  1. -1.]  At epoch: 2 Number of correctly classified patterns: 4
Final weights: [ 2.  1. -1.]  At epoch: 3 Number of correctly classified patterns: 3


The results match the ones we obtained by doing the calculations by hand.

In [10]:
# Load IRIS PLANT dataset.
df = pd.read_csv("./UCIdata-exercise1/iris.data", header = None, names = ["SL", "SP", "PL", "PW", "Class"])
df.head

<bound method NDFrame.head of       SL   SP   PL   PW           Class
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9  3.0  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4    5.0  3.6  1.4  0.2     Iris-setosa
..   ...  ...  ...  ...             ...
145  6.7  3.0  5.2  2.3  Iris-virginica
146  6.3  2.5  5.0  1.9  Iris-virginica
147  6.5  3.0  5.2  2.0  Iris-virginica
148  6.2  3.4  5.4  2.3  Iris-virginica
149  5.9  3.0  5.1  1.8  Iris-virginica

[150 rows x 5 columns]>

In [11]:
# Split dataset to matrices with each matrix containing the data for each type of plant.
X_set = df[df.Class=='Iris-setosa'].iloc[:, :4].values
X_ver = df[df.Class=='Iris-versicolor'].iloc[:, :4].values
X_vir = df[df.Class=='Iris-virginica'].iloc[:, :4].values

# Add -1 to the end of each sample.
minus_ones_set = -np.ones((X_set.shape[0], 1))
minus_ones_ver = -np.ones((X_ver.shape[0], 1))
minus_ones_vir = -np.ones((X_vir.shape[0], 1))

X_set = np.append(X_set, minus_ones_set, axis=1)
X_ver = np.append(X_ver, minus_ones_ver, axis=1)
X_vir = np.append(X_vir, minus_ones_vir, axis=1)

# Create 3 superclasses by concatenating every pair of classes.
X_set_ver = np.concatenate((X_set, X_ver), axis=0)
X_set_vir = np.concatenate((X_set, X_vir), axis=0)
X_ver_vir = np.concatenate((X_ver, X_vir), axis=0)

print(f"{X_set.shape}   {X_ver.shape}    {X_vir.shape}")
print(f"{X_set_ver.shape}   {X_set_vir.shape}    {X_ver_vir.shape}")

(50, 5)   (50, 5)    (50, 5)
(100, 5)   (100, 5)    (100, 5)


In [12]:
# Separating setosa from versicolor+virginica class.
w, epoch, count = perceptron(X_set, X_ver_vir, 1, -1, 10)
print(f"Final weights: {w}  At epoch: {epoch} Number of correctly classified patterns: {count}")

# Separating versicolor from setosa+virginica class.
w, epoch, count = perceptron(X_ver, X_set_vir, 1, -1, 1000)
print(f"Final weights: {w}  At epoch: {epoch} Number of correctly classified patterns: {count}")

# Separating virginica from setosa+versicolor class.
w, epoch, count = perceptron(X_vir, X_set_ver, 1, -1, 1000)
print(f"Final weights: {w}  At epoch: {epoch} Number of correctly classified patterns: {count}")

Final weights: [ 1.3  4.1 -5.2 -2.2 -1. ]  At epoch: 4 Number of correctly classified patterns: 150
Final weights: [ 38.8 -73.2   6.5 -64.  123. ]  At epoch: 1000 Number of correctly classified patterns: 143
Final weights: [-105.7 -128.1  150.5  244.5  180. ]  At epoch: 1000 Number of correctly classified patterns: 148


After running the perceptron algorithm on the dataset, it is clear that the setosa class is linearly separable from the other two classes. The algorithm has converged in a small number of epochs and the separating hyperplane is: $1.3\cdot x_1+4.1\cdot x_2 -5.2\cdot x_3 -2.2\cdot x_4 +1 =0$

As for the other two classes, they are not linearly separable from the other "super"classes. After running the algorithm 1000 epochs, we still have misclassified points (7 and 2 respectively). If we allow such a small margin of error (~5%) we can state that we indeed found a separaring hyperplane for each one of these two classes.